<a href="https://colab.research.google.com/github/kalehariprasad/sculpture-shopping-price/blob/main/notebook/customer_shopping_MLOPS_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [2]:
train_df=pd.read_csv('/content/drive/MyDrive/HackerEarth Machine Learning : Exhibit A(rt)/train.csv')


In [3]:
def ChangeDatatype(df):
  df['Scheduled Date']=pd.to_datetime(df['Scheduled Date'])
  df['Delivery Date']=pd.to_datetime(df['Delivery Date'])
  if 'Cost' in df.columns:
    df['Cost']=df['Cost'].abs()# for removing negitive sign from the cost column because cost has some values in negitive .but price will not be in negitive
  return df

In [4]:
train_df=ChangeDatatype(df=train_df)



In [5]:
def outliers_removal(df): #removing outliers
  new_df_cap = df.copy()

  for col in new_df_cap.columns:
      if new_df_cap[col].dtype != 'object' and new_df_cap[col].dtype != 'datetime64[ns]':
          percentile25 = new_df_cap[col].quantile(0.25)
          percentile75 = new_df_cap[col].quantile(0.75)
          iqr = percentile75 - percentile25
          upper_limit = percentile75 + 1.5 * iqr
          lower_limit = percentile25 - 1.5 * iqr

          new_df_cap[col] = np.where(
              new_df_cap[col] > upper_limit,
              upper_limit,
              np.where(
                  new_df_cap[col] < lower_limit,
                  lower_limit,
                  new_df_cap[col]
              )
          )

  return new_df_cap

In [6]:
train_df.head()

,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,2015-06-07,2015-06-03,"New Michelle, OH 50777",283.29
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,2017-03-06,2017-03-05,"New Michaelport, WY 12072",159.96
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,2015-03-09,2015-03-08,"Bowmanshire, WA 19241",154.29
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,2015-05-24,2015-05-20,"East Robyn, KY 86375",161.16
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,2016-12-18,2016-12-14,"Aprilside, PA 52793",159.23


In [7]:
train_df.columns

Index(['Customer Id', 'Artist Name', 'Artist Reputation', 'Height', 'Width',
       'Weight', 'Material', 'Price Of Sculpture', 'Base Shipping Price',
       'International', 'Express Shipment', 'Installation Included',
       'Transport', 'Fragile', 'Customer Information', 'Remote Location',
       'Scheduled Date', 'Delivery Date', 'Customer Location', 'Cost'],
      dtype='object')

In [8]:
train_df1=outliers_removal(df=train_df)

In [9]:
train_df1


,Customer Id,Artist Name,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,Scheduled Date,Delivery Date,Customer Location,Cost
0,fffe3900350033003300,Billy Jenkins,0.26,17.0,6.0,4128.0,Brass,13.91,16.27,Yes,Yes,No,Airways,No,Working Class,No,2015-06-07,2015-06-03,"New Michelle, OH 50777",283.29000
1,fffe3800330031003900,Jean Bryant,0.28,3.0,3.0,61.0,Brass,6.83,15.00,No,No,No,Roadways,No,Working Class,No,2017-03-06,2017-03-05,"New Michaelport, WY 12072",159.96000
2,fffe3600370035003100,Laura Miller,0.07,8.0,5.0,237.0,Clay,4.96,21.18,No,No,No,Roadways,Yes,Working Class,Yes,2015-03-09,2015-03-08,"Bowmanshire, WA 19241",154.29000
3,fffe350031003300,Robert Chaires,0.12,9.0,NaN,NaN,Aluminium,5.81,16.31,No,No,No,NaN,No,Wealthy,Yes,2015-05-24,2015-05-20,"East Robyn, KY 86375",161.16000
4,fffe3900320038003400,Rosalyn Krol,0.15,17.0,6.0,324.0,Aluminium,3.18,11.94,Yes,Yes,Yes,Airways,No,Working Class,No,2016-12-18,2016-12-14,"Aprilside, PA 52793",159.23000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,fffe3800370037003300,Jeffrey Freudenthal,0.37,37.0,10.0,16551.0,Brass,28.28,38.46,Yes,Yes,No,Airways,No,Wealthy,Yes,2018-03-28,2018-03-25,"New Robert, VT 85335",872.43000
6496,fffe310036003400,Larry Edwards,0.67,15.0,NaN,18981.0,NaN,67.18,27.72,No,No,No,Roadways,No,Working Class,No,2015-08-29,2015-08-27,"New Joshua, VA 35766",1347.02000
6497,fffe3600300031003300,Denise Worth,0.68,19.0,8.0,587.0,Clay,6.92,10.38,Yes,No,No,NaN,Yes,Wealthy,No,2019-04-10,2019-04-09,"Lake Kelly, MA 80823",354.55000
6498,fffe3600350035003900,Daniel Drew,0.02,33.0,9.0,90385.5,Stone,215.83,69.76,No,Yes,No,Roadways,No,Working Class,Yes,2019-03-10,2019-03-12,"Hintonberg, UT 35006",3075.48875


In [10]:
class DataProcessing:
    @staticmethod
    def remove_columns_outliers(df):
        # Column Removal
        excluded_columns = ['Customer Id', 'Artist Name', 'Width', 'Weight', 'Customer Location', 'Scheduled Date', 'Delivery Date']
        cleaned_df = df.drop(excluded_columns, axis=1)

        # Outliers Removal
        new_df = cleaned_df.copy()
        for col in new_df.columns:
            if new_df[col].dtype != 'object' and new_df[col].dtype != 'datetime64[ns]':
                percentile25 = new_df[col].quantile(0.25)
                percentile75 = new_df[col].quantile(0.75)
                iqr = percentile75 - percentile25
                upper_limit = percentile75 + 1.5 * iqr
                lower_limit = percentile25 - 1.5 * iqr
                new_df[col] = np.where((new_df[col] > upper_limit) | (new_df[col] < lower_limit), np.nan, new_df[col])

        return new_df.dropna()



In [11]:
class DataPreparation:
    def get_preprocessor(self):
        cat_pipe = Pipeline([
            ('imputation', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(handle_unknown='ignore', drop='first', sparse=False))
        ])
        num_pipe = Pipeline([
            ('imputation', SimpleImputer(strategy='mean')),
            ('scaling', StandardScaler())
        ])
        num_cols = ['Artist Reputation', 'Height', 'Price Of Sculpture', 'Base Shipping Price']
        cat_cols = ['Material', 'International', 'Express Shipment',
                    'Installation Included', 'Transport', 'Fragile', 'Customer Information',
                    'Remote Location']
        preprocessor = ColumnTransformer([
            ('cat_pipe', cat_pipe, cat_cols),
            ('num_pipe', num_pipe, num_cols)
        ], remainder='passthrough')

        return preprocessor

    def prepare_data(self, train_df, test_df):
        processed_train_df = DataProcessing.remove_columns_outliers(train_df)
        processed_test_df = DataProcessing.remove_columns_outliers(test_df)

        preprocessor_obj = self.get_preprocessor()

        # Preprocess both train and test DataFrames
        X_train = processed_train_df.drop("Cost", axis=1)
        X_train = preprocessor_obj.fit_transform(X_train)

        X_test = processed_test_df.drop("Cost", axis=1)
        X_test = preprocessor_obj.transform(X_test)

        y_train = processed_train_df['Cost']
        y_test = processed_test_df['Cost']

        return X_train, X_test, y_train, y_test


In [12]:
from sklearn.metrics import r2_score

class ModelTraining:
    @staticmethod
    def evaluate_models(X_train, y_train, X_test, y_test, models):
        report = {}
        best_model_name = None
        best_model_score = -1  # Initialize with a negative value

        for model_name, model in models.items():
            model.fit(X_train, y_train)  # Train model

            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)

            train_model_score = r2_score(y_train, y_train_pred)
            test_model_score = r2_score(y_test, y_test_pred)

            report[model_name] = test_model_score

            # Check if the current model is the best one based on the test score
            if test_model_score > best_model_score:
                best_model_score = test_model_score
                best_model_name = model_name

            print(f'Training score for {model_name}: {train_model_score}')
            print(f'Testing score for {model_name}: {test_model_score}')
            print('************************************************')

        return report, best_model_name


In [20]:
# Assuming train_df and test_df are your dataframes
data_prep = DataPreparation()
train_df,test_df=train_test_split(train_df1,test_size=0.3,random_state=42)
X_train, X_test, y_train, y_test = data_prep.prepare_data(train_df, test_df)

models = {
    "Linear Regression": LinearRegression(),
    "SVR": SVR(),
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "ExtraTreesRegressor": ExtraTreesRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

model_trainer = ModelTraining()



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
  models = {

                "linear Regression":LinearRegression(),
                "SVR":SVR(),
                "Random Forest": RandomForestRegressor(),
                "Decision Tree": DecisionTreeRegressor(),
                "ExtraTreesRegressor":ExtraTreesRegressor(),
                "Gradient Boosting": GradientBoostingRegressor(),

                }
param_grid = {
    "linear Regression": {},
    "SVR": {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"]
    },
    "Random Forest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    },
    "Decision Tree": {
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    },
    "ExtraTreesRegressor": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    },
    "Gradient Boosting": {
        "n_estimators": [100, 200, 300],
        "max_depth": [3, 4, 5],
        "learning_rate": [0.01, 0.1]
    }
}


In [17]:
def model_evaluation_hypertuning(X_train, y_train, X_test, y_test, models, param_grids):
    report = {}

    for model_name, model in models.items():
        param_grid = param_grids.get(model_name, {})  # Get the parameter grid for the model
        grid_search = GridSearchCV(model, param_grid, scoring='r2', cv=5)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_  # Get the best-tuned model
        y_train_pred = best_model.predict(X_train)
        y_test_pred = best_model.predict(X_test)

        train_model_score = r2_score(y_train, y_train_pred)
        test_model_score = r2_score(y_test, y_test_pred)

        report[model_name] = test_model_score

    return report


In [23]:
print('without hyperaparameters')
report, best_model = model_trainer.evaluate_models(X_train, y_train, X_test, y_test, models)

print("Best Model:", best_model)
print("Report:", report)


without hyperaparameters
Training score for Linear Regression: 0.8426758606708273
Testing score for Linear Regression: 0.5501198807278549
************************************************
Training score for SVR: 0.02603013329088888
Testing score for SVR: 0.364667282265531
************************************************
Training score for Random Forest: 0.9908181548310386
Testing score for Random Forest: 0.8802495667854682
************************************************
Training score for Decision Tree: 1.0
Testing score for Decision Tree: 0.6098925684169575
************************************************
Training score for ExtraTreesRegressor: 1.0
Testing score for ExtraTreesRegressor: 0.8747798718803604
************************************************
Training score for Gradient Boosting: 0.9526833036623159
Testing score for Gradient Boosting: 0.8625791601509576
************************************************
Best Model: Random Forest
Report: {'Linear Regression': 0.550119880727854

In [25]:
report=model_evaluation_hypertuning(X_train, y_train, X_test, y_test, models, param_grid)


In [27]:
print('with hyperparameter tuninhg')
report

with hyperparameter tuninhg


{'Linear Regression': 0.5501198807278549,
 'SVR': 0.4960817787313173,
 'Random Forest': 0.8843762425249926,
 'Decision Tree': 0.6597996425581895,
 'ExtraTreesRegressor': 0.8766955606999755,
 'Gradient Boosting': 0.9062050963734007}